In [2]:
import os
os.chdir('../quafing/')
print(f"Working directory: {os.getcwd()}")
import quafing as q


import random
import numpy as np
from scipy.stats import mode

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import matplotlib.pyplot as plt


plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({"font.size" : 15, 
                     "figure.dpi" : 100, 
                     "legend.fontsize" : 13, 
                     "grid.alpha" : 0.3, 
                     "axes.grid": True, 
                     "axes.axisbelow" : True, 
                     "figure.figsize":(6, 5)})

Working directory: /Users/charlesdupont/Desktop/Thesis/code/quafing


In [424]:
def load_data(path):
    """
    Loads .dta file using provided path.
    """
    return pd.read_stata(path, convert_categoricals=False)


def add_missing_hhids(all_hhids, df):
    """
    Adds empty rows for hhids missing from df.
    df must already have an "hhid" column!
    """
    missing = set(unique_hhids) - set(df["hhid"].unique())
    missing_dict = {"hhid":list(missing)}
    for col in df.columns:
        if col != "hhid":
            missing_dict[col] = [np.nan]*len(missing)
    missing_df = pd.DataFrame.from_dict(missing_dict)
    return pd.concat([df, missing_df])


def column_selection(df, keep=None, remove=None):
    """
    Performs column selection on dataframe.
    Args:
        no_change: boolean indicating whether to keep all columns
        keep: list of columns to keep
        remove: list of columns to remove
    Returns:
        updated dataframe
    """
    if keep:
        return df[keep]
    elif remove:
        return df.loc[:, ~df.columns.isin(remove)]
    else:
        return df

In [394]:
DATA_DIR = "../../BCCASII/"
DATA_SUBDIR = "Household/"

## A: Location and identification detail

In [396]:
filename = "001_mod_a.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [397]:
# keep household ID, agro-ecological zone
processed = column_selection(df, keep=["hhid", "ezcode"])

# construct community code from union, thana, district codes
processed["community_code"] = df["ucode"].astype(str) + "_" + \
                              df["tcode"].astype(str) + "_" + \
                              df["dcode"].astype(str)

In [399]:
unique_hhids = df["hhid"].unique()

## B: Household composition and education

In [401]:
filename = "002_mod_b.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [412]:
g = df.groupby("hhid").count()

# household size
processed = pd.DataFrame.from_dict({"household_size":g["mid"]})

# household id
processed["hhid"] = processed.index

# ratio male, female
males = {}
females = {}
for hhid in unique_hhids:
    household = df.loc[df.hhid==hhid]
    m = sum(household["b01"]==1)
    f = sum(household["b01"]==2)
    s = max(1, m+f)
    males[hhid] = m/s
    females[hhid] = f/s
processed = processed.join(pd.DataFrame.from_dict({"ratio_male":males}))
processed = processed.join(pd.DataFrame.from_dict({"ratio_female":females}))

# number of children
num_children = {}
for hhid in unique_hhids:
    household = df.loc[df.hhid==hhid]
    c = sum(household["b03"] < 18)
    num_children[hhid] = c
processed = processed.join(pd.DataFrame.from_dict({"number_children":num_children}))

# highest level of education
education_ordering = [99, 66, 67, 68, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 22, 10, 33, 12, 15, 16, 17]
max_education = {}
for hhid in unique_hhids:
    household = df.loc[df.hhid==hhid]
    education_levels = [education_ordering.index(x)
                        for x in [x for x in list(household["b06"]) if not np.isnan(x)]]
    if len(education_levels):
        max_education[hhid] = max(education_levels)
    else:
        max_education[hhid] = -1
processed = processed.join(pd.DataFrame.from_dict({"highest_education":max_education}))

# highest level of literacy
literacy_ordering = [1, 2, 3, 4]
max_literacy = {}
for hhid in unique_hhids:
    household = df.loc[df.hhid==hhid]
    literacy_labels = [literacy_ordering.index(x)
                       for x in [x for x in list(household["b05"]) if not np.isnan(x)]]
    if len(literacy_labels):
        max_literacy[hhid] = max(literacy_labels)
    else:
        max_literacy[hhid] = -1
processed = processed.join(pd.DataFrame.from_dict({"highest_literacy":max_literacy}))

# primary occupation categories
# categories = {
#     "day_labor": [1,2,3],
#     "self_employ_farm": [4,5,6,7,8],
#     "self_employ_non_farm": [9],
#     "low_income_profession": [10,11,12,13],
#     "mechanic": [14,15],
#     "other": [16, 17, 18, 19],
#     "non_earning": [20,21,22,23,24,25,26]
# }

# for c, c_list in categories.items():
#     c_dict = {hhid:0 for hhid in unique_hhids}
#     for hhid in unique_hhids:
#         household = df.loc[df.hhid==hhid]
#         primary_occupation_labels = list(household["b08_1"])
#         for item in c_list:
#             if item in primary_occupation_labels:
#                 c_dict[hhid] += 1
#     processed = processed.join(pd.DataFrame.from_dict({c:c_dict}))

## C: Roster of land and water bodies owned or under operation

In [418]:
filename = "003_mod_c.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [425]:
g = df.groupby("hhid")

# total land area
processed = pd.DataFrame.from_dict({"total_land_size":dict(g.sum()["c102"])})

# household id
processed["hhid"] = processed.index

# total value per month
processed = processed.join(pd.DataFrame.from_dict({"total_land_value_per_month":
                                                   dict(g.sum()["c113"])}))

# min land acquisition year
processed = processed.join(pd.DataFrame.from_dict({"min_land_acquisition_year":
                                                   dict(g.min()["c111"])}))

# max land acquisition year
processed = processed.join(pd.DataFrame.from_dict({"max_land_acquisition_year":
                                                   dict(g.max()["c111"])}))

# mean land acquisition year
processed = processed.join(pd.DataFrame.from_dict({"mean_land_acquisition_year":
                                                   dict(round(g.mean()["c111"]))}))

# most common answer from (plot type, soil type, operational status of land, acquisition means of land)
categories = {
    ("plot_type", "c101"): [i for i in range(1, 10)],
    ("soil_type", "c105"): [i for i in range(1, 6)],
    ("land_operational_status", "c107"): [i for i in range(1, 11)],
    ("land_acquisition_means", "c110"): [i for i in range(1, 8)],
}

for c, c_list in categories.items():
    c_dict = {hhid:0 for hhid in unique_hhids}
    c, col = c
    for hhid in unique_hhids:
        household = df.loc[df.hhid==hhid]
        labels = [x for x in list(household[col]) if not np.isnan(x)]
        if labels:
            c_dict[hhid] = mode(labels)[0][0]
        else:
            c_dict[hhid] = -1
    processed = processed.join(pd.DataFrame.from_dict({c:c_dict}))

## F1: Livestock and poultry

In [428]:
filename = "004_mod_f1.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [436]:
# household id
processed = pd.DataFrame.from_dict({"hhid":list(unique_hhids)})

categories = {
    "cattle_buffalo":1,
    "goat_sheep":2,
    "pigs":3,
    "chidken_duck":4
}

cols = [("count_2011", "f102"), ("value_2011", "f103"), 
        ("count_now", "f104"), ("value_now", "f105"), ("total_selling_value", "f117")]   

results = {hhid:{} for hhid in unique_hhids}
for hhid in unique_hhids:
    household = df.loc[df.hhid==hhid]
    for c, number in categories.items():
        if number not in list(household["f101"]):
            # TODO: handle missing value!
            for l, _ in cols:
                results[hhid][c+"_"+l] = np.nan
        else:
            for l, col in cols:
                value = household.loc[household["f101"]==number][col].iloc[0]
                # TODO: handle missing value here!
                if np.isnan(value):
                    results[hhid][c+"_"+l] = np.nan
                else:
                    results[hhid][c+"_"+l] = value
                    
for column in results[1]:
    processed[column] = [results[hhid][column] for hhid in results]

## F3: Expenditure for livestock, Poultry/Duckery Production Last 12 Months

In [443]:
filename = "005_mod_f3.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [445]:
df["f306"] = df["f306_1"].fillna(0) + df["f306_2"].fillna(0)
df = df.groupby("hhid").sum()
processed = column_selection(df, keep=["f302", "f303", "f306", "f307"])
processed["hhid"] = processed.index

## G1: Credit obtained

In [449]:
filename = "006_mod_g.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [450]:
df = df.groupby("hhid").sum()
processed = column_selection(df, keep=["g07", "g10", "g16", "g18"])
processed["hhid"] = processed.index

## G2: Repayment and utilization of Loan

In [ ]:
# TODO

## H: Household Assets

In [453]:
filename = "007_mod_h.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [454]:
# household id
processed = pd.DataFrame.from_dict({"hhid":list(unique_hhids)})

# total value per household asset category
categories = {
    "value_consumable_durables":[i for i in range(1, 12)],
    "value_transport":[i for i in range(12, 16)],
    "value_livestock_poultry":[i for i in range(16, 20)],
    "value_agricultural_equipments":[i for i in range(20,29)],
    "value_other":[29]
}

results = {hhid:{} for hhid in unique_hhids}
for hhid in unique_hhids:
    household = df.loc[df.hhid==hhid]
    for c in categories:
        category_assets = household.loc[household["asset"].isin(categories[c])]
        if len(category_assets):
            results[hhid][c] = category_assets["h06"].sum()
        else:
            results[hhid][c] = 0

for column in results[1]:
    processed[column] = [results[hhid][column] for hhid in results]

## H1: Assets: Sales of Assets and Change in Ownership

In [459]:
filename = "008_mod_h1.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [460]:
df = df.groupby("hhid").sum()
processed = column_selection(df, keep=["h108"])
processed["hhid"] = processed.index

## I: Food Consumption: Purchases, Home Production, and Other Sources

In [461]:
filename = "009_mod_i.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [462]:
df = df.groupby("hhid").sum()
processed = column_selection(df, keep=["i07"])
processed["hhid"] = processed.index

## J1, J2: Non-food Expenditure

In [464]:
filename = "010_mod_j.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [465]:
df = df.groupby("hhid").sum()
procesed = column_selection(df, keep=["j02_2", "j03_2"])
processed["hhid"] = processed.index

## K: Housing, Source of Water and sanitation

In [466]:
filename = "011_mod_k.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [467]:
processed = column_selection(df, keep=["hhid", "k01", "k02", "k03", "k03_1", 
                                       "k04", "k05", "k05_1", "k06", "k07", "k10"])

## L : Weather events adversely affected the household or the farming

In [469]:
filename = "012_mod_l.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [470]:
calamity_numbers = [i for i in range(1, 13)]

results = {hhid:{} for hhid in unique_hhids}

for hhid in unique_hhids:
    household = df.loc[df.hhid == hhid]
    nums = list(household["l01"])
    for n in calamity_numbers:
        # TODO: missing values!
        if n not in nums:
            results[hhid]["calamity_"+str(n)] = 0
        else:
            if int(household.loc[household["l01"]==n]["l02"]) == 1:
                results[hhid]["calamity_"+str(n)] = 1
            else:
                results[hhid]["calamity_"+str(n)] = 0       

processed = pd.DataFrame()
for column in results[1]:
    processed[column] = [results[hhid][column] for hhid in results]
    
processed["hhid"] = unique_hhids

## L1: Weather events adversely affected the household or the farming cope

In [471]:
filename = "013_mod_l1.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [472]:
df = df.groupby("hhid").sum()
processed = column_selection(df, keep=["l1_05"])
processed["hhid"] = processed.index

## L2: Shocks

In [474]:
filename = "014_mod_l2.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [475]:
df = df.groupby("hhid").sum()
processed = column_selection(df, keep=["l2_06"])
processed["hhid"] = processed.index

## L3: Positive Economic Events

In [476]:
filename = "015_mod_l3.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [477]:
df = df.groupby("hhid").sum()
processed = column_selection(df, keep=["l3_06"])
processed["hhid"] = processed.index

## M: Agricultural extension

In [482]:
filename = "017_mod_m.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [483]:
processed = column_selection(df)

## Q: Perceptions of Climate Change

In [486]:
filename = "018_mod_q.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [487]:
processed = column_selection(df)

## O: Changes in farming practices due to climate change

In [489]:
filename = "019_mod_o.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [490]:
keep = ["o0"+str(i)+"_1" for i in range(2, 9)] + ["o09"] + \
       ["o"+str(i) for i in range(10, 23)] + ["o"+str(i)+"_1" for i in range(23, 29)] + ["o29"]

processed = column_selection(df, keep=keep)

## O1: Any integration about Livestock/Poultry

In [492]:
filename = "020_mod_o1.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [496]:
keep = ["o1_01_"+str(i) for i in range(1, 10)]
processed = column_selection(df, keep=keep)

## O2: Constraints to adaptation

In [497]:
filename = "021_mod_o2.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [498]:
# TODO

## R1: Information of group based approach

In [499]:
filename = "022_mod_r1.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [500]:
groups = np.arange(1, 19)

results = {hhid:{} for hhid in unique_hhids}
for hhid in unique_hhids:
    household = df.loc[df.hhid==hhid]
    for g in groups:
        g_row = household.loc[household["r1_01"]==g]
        if len(g_row):
            membership_values = list(g_row["r1_02"])
            if 1 in membership_values:
                results[hhid][f"group_{g}_membership"] = 1
            else:
                results[hhid][f"group_{g}_membership"] = 0
        else:
            results[hhid][f"group_{g}_membership"] = 0
            
processed = pd.DataFrame()
for column in results[1]:
    processed[column] = [results[hhid][column] for hhid in results]
    
processed["hhid"] = unique_hhids

## R2: Group activity related information

In [508]:
# TODO: skip?

## R3: Network

In [509]:
filename = "024_mod_r3.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [510]:
processed = column_selection(df, remove=["r3_01_1", "r3_01_2"])

## R4: Trust and Solidarity

In [514]:
filename = "025_mod_r4.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [515]:
processed = column_selection(df, remove=["r4_01_1", "r4_01_2"])

## R5: Economic setbacks

In [553]:
filename = "026_mod_r5.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [574]:
results = {hhid:0 for hhid in unique_hhids}
for hhid in unique_hhids:
    household = df.loc[df.hhid==hhid]
    if 1 in list(household["r5"]):
        results[hhid] = 1
        
processed = pd.DataFrame.from_dict({"r5":results})
processed["hhid"] = processed.index

## R6: Collective Action and Cooperation and Sociability

In [577]:
filename = "027_mod_r6.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [578]:
processed = column_selection(df, remove=["r6_01_1", "r6_01_2"])

## R7: Information and Communication

In [581]:
filename = "028_mod_r7.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [582]:
processed = column_selection(df, remove=["r7_01_1", "r7_01_2"])

## R8: Production and climate Information

In [583]:
filename = "029_mod_r8.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [584]:
keep = ["hhid", "r8_02_1", "r8_02_2", "r8_06_1", "r8_06_2", "r8_14_1", "r8_14_2"]
processed = column_selection(df, keep=keep)

## R9: Production and climate Information

In [586]:
filename = "030_mod_r9.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [587]:
keep = ["hhid", "r9_02_1", "r9_02_2", "r9_09_1", "r9_09_2"]
processed = column_selection(df, keep=keep)

## S: Empowerment and Political Action

In [588]:
filename = "031_mod_s.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [589]:
processed = column_selection(df, remove=["s_01_1", "s_01_2"])

## T: Employment

In [590]:
filename = "032_mod_t.dta"
path = DATA_DIR + DATA_SUBDIR + filename
df = load_data(path)

In [591]:
job_categories = {
    "job_wage_labor":np.arange(1,12),
    "job_salaried_worker":np.arange(12,22),
    "job_self_employment":np.arange(22,48),
    "job_trader":np.arange(50,55),
    "job_production":np.arange(55,58),
    "job_livestock_poultry_service":np.arange(58,64),
    "job_farming":np.arange(64,73)
}

results = {hhid:{c:0 for c in job_categories} for hhid in unique_hhids}
for hhid in unique_hhids:
    household = df.loc[df.hhid==hhid]
    relevant = list(household["t05"])
    for c, numbers in job_categories.items():
        for n in numbers:
            if n in relevant:
                results[hhid][c] += 1
            
processed = pd.DataFrame.from_dict({"hhid":list(unique_hhids)})        
for column in results[1]:
    processed[column] = [results[hhid][column] for hhid in results]

## U: Time use of male and female members in the household

In [592]:
# TODO: discard?

## V: Anthropometry

In [593]:
# TODO: discard?

# Combine processed dataframes

In [186]:
# maintain list of all the processed dataframes
# make sure all dataframes have one row per household id
# add empty rows for missing household ids where necessary
# mind the treatment of MISSING VALUES!
    # do we really want to impute "0" when no information is present?
# combine all the dataframes into one dataframe (each hhid is a single feature vector/row)